In [29]:
import os

import chromadb
from openai import OpenAI
from dotenv import load_dotenv
from IPython.display import Markdown

In [2]:
load_dotenv()

True

In [3]:
client = OpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
)

model = "gpt-3.5-turbo"

In [5]:
chroma_db = chromadb.PersistentClient(path="../4_vector_databases/db")

chroma_db.list_collections()

[Collection(name=movies)]

In [19]:
chroma_collection = chroma_db.get_or_create_collection("movies")
res = chroma_collection.query(query_texts=["wolverine"], n_results=3)
res

{'ids': [['288410', '320288', '263115']],
 'distances': [[0.9013055562973022, 1.0991826057434082, 1.1146235466003418]],
 'metadatas': [[{'source': "Chris Claremont's X-Men"},
   {'source': 'Dark Phoenix'},
   {'source': 'Logan'}]],
 'embeddings': None,
 'documents': [['Chris Claremont’s X-Men takes an in-depth look at Claremont’s monumental run. Using high-profile interviews the film explores the behind-the-scenes development of notable characters like Wolverine Storm and Phoenix as well as the challenges of creating art within a corporate system.',
   "The X-Men face their most formidable and powerful foe when one of their own Jean Grey starts to spiral out of control. During a rescue mission in outer space Jean is nearly killed when she's hit by a mysterious cosmic force. Once she returns home this force not only makes her infinitely more powerful but far more unstable. The X-Men must now band together to save her soul and battle aliens that want to use Grey's new abilities to rule t

In [20]:
len(chroma_collection.get()["ids"])

14501

In [23]:
def get_query_results(query_text: str, n_results: int = 5):
    res = chroma_collection.query(query_texts=[query_text], n_results=3)
    docs = res["documents"][0]
    titles = [item["source"] for item in res["metadatas"][0]]
    res_string = ";".join(
        [f"{title}: {description}" for title, description in zip(titles, docs)]
    )
    return res_string

In [26]:
query_text = "wolverine"
retrieved_results = get_query_results(query_text)
print(retrieved_results)

Chris Claremont's X-Men: Chris Claremont’s X-Men takes an in-depth look at Claremont’s monumental run. Using high-profile interviews the film explores the behind-the-scenes development of notable characters like Wolverine Storm and Phoenix as well as the challenges of creating art within a corporate system.;Dark Phoenix: The X-Men face their most formidable and powerful foe when one of their own Jean Grey starts to spiral out of control. During a rescue mission in outer space Jean is nearly killed when she's hit by a mysterious cosmic force. Once she returns home this force not only makes her infinitely more powerful but far more unstable. The X-Men must now band together to save her soul and battle aliens that want to use Grey's new abilities to rule the galaxy.;Logan: In the near future a weary Logan cares for an ailing Professor X in a hideout on the Mexican border. But Logan's attempts to hide from the world and his legacy are upended when a young mutant arrives pursued by dark for

In [27]:
system_role_definition = "You are a an expert in movies. Users will ask you questions about movies. You will get a user question, and relevant information. Relevant information is structured like movie title:movie plot; ... Please answer the question only using the information provided. format your response in the form of a markdown table"
user_query = "What are the names of the movies and their plot where {user_query}?"
messages = [
    {"role": "system", "content": system_role_definition},
    {"role": "user", "content": f"{user_query}; \n Information: {retrieved_results}"},
]

In [30]:
response = client.chat.completions.create(model=model, messages=messages)

content = response.choices[0].message.content

In [31]:
Markdown(content)

| Movie Title              | Plot                                                                                                                                                                                                                                                                                                      |
|--------------------------|-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Dark Phoenix             | The X-Men face their most formidable and powerful foe when one of their own Jean Grey starts to spiral out of control. During a rescue mission in outer space Jean is nearly killed when she's hit by a mysterious cosmic force. Once she returns home this force not only makes her infinitely more powerful but far more unstable. The X-Men must now band together to save her soul and battle aliens that want to use Grey's new abilities to rule the galaxy. |
| Logan                    | In the near future, a weary Logan cares for an ailing Professor X in a hideout on the Mexican border. But Logan's attempts to hide from the world and his legacy are upended when a young mutant arrives pursued by dark forces.                                                                                                                                               |

In [32]:
def rag(user_query: str):
    retrieved_results = get_query_results(user_query)
    system_role_definition = "You are a an expert in movies. Users will ask you questions about movies. You will get a user question, and relevant information. Relevant information is structured like movie title:movie plot; ... Please answer the question only using the information provided. format your response in the form of a markdown table"
    user_query_complete = (
        f"What are the names of the movies and their plot where {user_query}?"
    )
    messages = [
        {"role": "system", "content": system_role_definition},
        {
            "role": "user",
            "content": f"{user_query_complete}; \n Information: {retrieved_results}",
        },
    ]
    response = client.chat.completions.create(model=model, messages=messages)
    content = response.choices[0].message.content
    return content

In [33]:
print("Response from Vector DB")

query = "superhero spider"
print(get_query_results(query))

Response from Vector DB
Spider-Man: Across the Spider-Verse: After reuniting with Gwen Stacy Brooklyn’s full-time friendly neighborhood Spider-Man is catapulted across the Multiverse where he encounters the Spider Society a team of Spider-People charged with protecting the Multiverse’s very existence. But when the heroes clash on how to handle a new threat Miles finds himself pitted against the other Spiders and must set out on his own to save those he loves most.;Spider-Man: Homecoming: Following the events of Captain America: Civil War Peter Parker with the help of his mentor Tony Stark tries to balance his life as an ordinary high school student in Queens New York City with fighting crime as his superhero alter ego Spider-Man as a new threat the Vulture emerges.;Spider-Man: Into the Spider-Verse: Miles Morales is juggling his life between being a high school student and being a spider-man. When Wilson "Kingpin" Fisk uses a super collider others from across the Spider-Verse are trans

In [35]:
print("Response from RAG")

Markdown(rag(query))

Response from RAG


| Movie Title                      | Movie Plot                                                                                                                                                                                                                                                                    |
|----------------------------------|--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| Spider-Man: Across the Spider-Verse | After reuniting with Gwen Stacy Brooklyn’s full-time friendly neighborhood Spider-Man is catapulted across the Multiverse where he encounters the Spider Society a team of Spider-People charged with protecting the Multiverse’s very existence. |
| Spider-Man: Homecoming                | Following the events of Captain America: Civil War Peter Parker with the help of his mentor Tony Stark tries to balance his life as an ordinary high school student in Queens New York City with fighting crime as his superhero alter ego Spider-Man as a new threat the Vulture emerges.                             |
| Spider-Man: Into the Spider-Verse     | Miles Morales is juggling his life between being a high school student and being a spider-man. When Wilson "Kingpin" Fisk uses a super collider others from across the Spider-Verse are transported to this dimension.                                                                    |